In [18]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import math

random.seed(1)
torch.manual_seed(1)

import pickle

In [19]:
# Lineups format
#[(home_lineup, away_lineup, box_stats, (num_posessions, score_update))]
# Box stats format
# 10x13 array
# ['PTS', 'TS%', 'FGA', 'AST', 'REB', 'STL', 'BLK', '2PA', '2PM', '3PA', '3PM', 'FTA', 'FTM']
# for each player (home & away)
with open("processed_data/pickled_lineups1998-2018",'rb') as infile:
    lineups = pickle.load(infile)
    
with open("processed_data/lineup_metadata1998-2018",'rb') as infile:
    player_to_idx, idx_to_player, normal_name_to_idx, all_stats = pickle.load(infile)

In [20]:
def random_order(dset, batch_size=64):
    order = [i for i in range(len(dset))]
    random.shuffle(order)
    o_b, d_b, b_b, n_b, s_b = [], [], [], [], []
    samples = 0
    for i, o in enumerate(order):
        offense, defense, box, (n_pos, score_change) = dset[o]
        if len(offense) == 5 and len(defense) == 5:
            o_b.append(offense)
            d_b.append(defense)
            b_b.append(box)
            n_b.append(n_pos)
            s_b.append(score_change)
            samples += 1
        if samples == batch_size:
            yield o_b, d_b, b_b, n_b, s_b
            o_b, d_b, b_b, n_b, s_b = [], [], [], [], []
            samples = 0

In [21]:
tot_pts = 0
tot_pos = 0
for lineup in lineups:
    pos, pts = lineup[3]
    tot_pos += pos
    tot_pts += pos * pts
avg_ppp = tot_pts/tot_pos

In [22]:
class Rapm_Model(nn.Module):
    def __init__(self, n_players):
        super(Rapm_Model, self).__init__()
        # Create 1-d embeddings for players (pseudo-RAPM)
        self.offensive_embeddings = nn.Embedding(n_players,1)
        self.offensive_embeddings.weight = nn.Parameter(torch.zeros(n_players,1))
        
        self.defensive_embeddings = nn.Embedding(n_players,1)
        self.defensive_embeddings.weight = nn.Parameter(torch.zeros(n_players,1))
        
    def forward(self, inputs):
        o = self.offensive_embeddings(inputs[0])
        d = self.defensive_embeddings(inputs[1])
        return torch.sum(o, dim=1) - torch.sum(d, dim=1)

In [23]:
class Rapm_Model(nn.Module):
    def __init__(self, n_players):
        super(Rapm_Model, self).__init__()
        # Create 1-d embeddings for players (pseudo-RAPM)
        self.offensive_embeddings = nn.Embedding(n_players,1)
        self.offensive_embeddings.weight = nn.Parameter(torch.zeros(n_players,1))
        
        self.defensive_embeddings = nn.Embedding(n_players,1)
        self.defensive_embeddings.weight = nn.Parameter(torch.zeros(n_players,1))
        
    def forward(self, inputs):
        o = self.offensive_embeddings(inputs[0])
        d = self.defensive_embeddings(inputs[1])
        return torch.sum(o, dim=1) - torch.sum(d, dim=1)

In [24]:
death_lineup = ['stephen curry', 'draymond green', 'klay thompson', 'andre iguodala', 'kevin durant']
death_lineup = [[normal_name_to_idx[name] for name in death_lineup]]
print(death_lineup)
dead_lineup = ['ian clark', 'david west', 'shaun livingston', 'patrick mccaw', 'zaza pachulia']
dead_lineup = [[normal_name_to_idx[name] for name in dead_lineup]]
death_lineup = torch.LongTensor(death_lineup).cuda()
dead_lineup = torch.LongTensor(dead_lineup).cuda()
print(dead_lineup)

[[554, 551, 553, 38, 56]]
tensor([[825,  90, 454, 890,  14]], device='cuda:0')


In [25]:
model = Rapm_Model(len(idx_to_player)).cuda()
epochs = 50
batch_size = 1 
loss_function = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.01)


In [ ]:
losses = []
for i in range(epochs):
    total_loss = 0
    for i, lineup in enumerate(random_order(lineups, batch_size=batch_size)):

        offense, defense, box, n_pos, score_change = lineup
        # Normalize the score change
        score_change = (torch.tensor(score_change).cuda() - avg_ppp) * 100
        
        offense = torch.LongTensor(offense).cuda()
        defense = torch.LongTensor(defense).cuda()
        n_pos = torch.FloatTensor(n_pos).cuda()
        model.zero_grad()

        p_score_change = model((offense, defense))
        loss = (p_score_change - score_change)**2 * n_pos
        loss = torch.sum(loss) / batch_size
        
        loss.backward()
        optimizer.step()
        

        total_loss += loss.item()
        if i % 10000 == 0:
            print()
            print("starters: ",model((death_lineup, dead_lineup)).item())
            print("bench: ",model((dead_lineup, death_lineup)).item())
            print("Curry: ", 
                  model.offensive_embeddings(torch.LongTensor([normal_name_to_idx['stephen curry']]).cuda())[0].item()
                 )
            print("KD: ",
                  model.offensive_embeddings(torch.LongTensor([normal_name_to_idx['kevin durant']]).cuda())[0].item()
                 )
            print("Lebron: ",
                  model.offensive_embeddings(torch.LongTensor([normal_name_to_idx['lebron james']]).cuda())[0].item()
                 )
            print("McCaw: ",
                  model.offensive_embeddings(torch.LongTensor([normal_name_to_idx['patrick mccaw']]).cuda())[0].item()
                 )
    losses.append(math.sqrt(total_loss/tot_pos))
    
print(losses)  # The loss decreased every iteration over the training data!

    


starters:  0.0
bench:  0.0
Curry:  0.0
KD:  0.0
Lebron:  0.0
McCaw:  0.0

starters:  3.3525524139404297
bench:  4.00864839553833
Curry:  0.6991000175476074
KD:  0.62544184923172
Lebron:  2.39377498626709
McCaw:  -0.4382564425468445

starters:  6.60927677154541
bench:  3.799825668334961
Curry:  1.2754571437835693
KD:  1.0570005178451538
Lebron:  2.365553379058838
McCaw:  0.6665157675743103

starters:  7.910127639770508
bench:  1.3658695220947266
Curry:  2.222114324569702
KD:  2.751349687576294
Lebron:  2.538778066635132
McCaw:  -0.36130276322364807

starters:  13.82052993774414
bench:  2.529336452484131
Curry:  2.7860732078552246
KD:  4.2856292724609375
Lebron:  3.4191508293151855
McCaw:  0.4632014334201813

starters:  14.638839721679688
bench:  1.0359899997711182
Curry:  3.748626232147217
KD:  5.42738676071167
Lebron:  3.449397563934326
McCaw:  1.2777574062347412

starters:  14.06166934967041
bench:  -1.2313957214355469
Curry:  3.836629629135132
KD:  3.921626329421997
Lebron:  4.32238


starters:  12.163241386413574
bench:  -10.219460487365723
Curry:  5.3515849113464355
KD:  3.1412572860717773
Lebron:  6.424478054046631
McCaw:  0.483228862285614

starters:  19.15593719482422
bench:  -5.173628807067871
Curry:  6.721931457519531
KD:  3.691094160079956
Lebron:  5.124120712280273
McCaw:  0.7875239253044128

starters:  20.360183715820312
bench:  -9.51883602142334
Curry:  7.07874870300293
KD:  4.454765796661377
Lebron:  5.196691989898682
McCaw:  0.5382804870605469

starters:  16.04623031616211
bench:  -6.183335304260254
Curry:  6.006505489349365
KD:  4.293669700622559
Lebron:  5.371940612792969
McCaw:  1.3230409622192383

starters:  14.19436264038086
bench:  -6.272870063781738
Curry:  6.227948188781738
KD:  5.13301420211792
Lebron:  4.171611785888672
McCaw:  1.0624604225158691

starters:  14.963740348815918
bench:  -9.473170280456543
Curry:  6.990969181060791
KD:  4.819488525390625
Lebron:  4.233458995819092
McCaw:  0.8249570727348328

starters:  16.008281707763672
bench: 


starters:  20.22507667541504
bench:  -5.69455623626709
Curry:  8.444534301757812
KD:  4.405944347381592
Lebron:  4.768491744995117
McCaw:  -0.732912003993988

starters:  18.281822204589844
bench:  -10.496170043945312
Curry:  8.395341873168945
KD:  4.380348205566406
Lebron:  5.821357727050781
McCaw:  -1.6440564393997192

starters:  13.465627670288086
bench:  -10.103538513183594
Curry:  7.548357009887695
KD:  4.0143351554870605
Lebron:  5.009057998657227
McCaw:  -1.4314979314804077

starters:  17.20587921142578
bench:  -10.288421630859375
Curry:  7.42466402053833
KD:  3.8028016090393066
Lebron:  5.708580017089844
McCaw:  -1.258834958076477

starters:  22.142105102539062
bench:  -10.31639289855957
Curry:  7.227550506591797
KD:  2.356271266937256
Lebron:  6.622246265411377
McCaw:  -2.146199941635132

starters:  22.605449676513672
bench:  -6.07513952255249
Curry:  7.398314952850342
KD:  0.8727809190750122
Lebron:  5.99020528793335
McCaw:  -1.8763208389282227

starters:  16.687042236328125



starters:  17.929393768310547
bench:  -7.800500869750977
Curry:  7.140104293823242
KD:  4.495182037353516
Lebron:  5.098250389099121
McCaw:  0.021404238417744637

starters:  15.682125091552734
bench:  -12.09185791015625
Curry:  6.311526298522949
KD:  4.880242824554443
Lebron:  4.912374019622803
McCaw:  -1.04183828830719

starters:  11.855122566223145
bench:  -9.01697063446045
Curry:  6.319520473480225
KD:  4.092390060424805
Lebron:  5.206780433654785
McCaw:  -0.822340190410614

starters:  19.960262298583984
bench:  -9.400325775146484
Curry:  7.157999038696289
KD:  4.833631992340088
Lebron:  5.093374252319336
McCaw:  -0.8891883492469788

starters:  18.102128982543945
bench:  -5.162858963012695
Curry:  7.549149036407471
KD:  3.6407387256622314
Lebron:  5.512672424316406
McCaw:  -0.868628740310669

starters:  19.98220443725586
bench:  -8.509230613708496
Curry:  8.007059097290039
KD:  4.494582176208496
Lebron:  4.308070182800293
McCaw:  -0.6506969332695007

starters:  21.07024383544922
be


starters:  15.216140747070312
bench:  -9.541091918945312
Curry:  6.136006832122803
KD:  3.1858716011047363
Lebron:  5.5365800857543945
McCaw:  0.2560148537158966

starters:  19.015851974487305
bench:  -7.3891096115112305
Curry:  7.617363452911377
KD:  2.3710880279541016
Lebron:  4.848069667816162
McCaw:  0.5326231122016907

starters:  22.880441665649414
bench:  -6.751877784729004
Curry:  8.207097053527832
KD:  3.6755197048187256
Lebron:  4.954461097717285
McCaw:  1.1128724813461304

starters:  21.206340789794922
bench:  -12.836742401123047
Curry:  7.300876140594482
KD:  4.324798107147217
Lebron:  4.864267349243164
McCaw:  0.009902800433337688

starters:  17.74243927001953
bench:  -16.627098083496094
Curry:  8.047820091247559
KD:  3.0884883403778076
Lebron:  5.910171031951904
McCaw:  -0.691641628742218

starters:  13.217151641845703
bench:  -13.469948768615723
Curry:  7.492591857910156
KD:  2.49509334564209
Lebron:  6.474070072174072
McCaw:  -1.253123164176941

starters:  14.3581180572

...


starters:  24.961713790893555
bench:  -16.64037322998047
Curry:  9.956676483154297
KD:  3.4707887172698975
Lebron:  6.911709308624268
McCaw:  -2.5777108669281006

starters:  20.413984298706055
bench:  -10.032686233520508
Curry:  8.879829406738281
KD:  4.7496209144592285
Lebron:  6.409190654754639
McCaw:  -2.3348162174224854

starters:  19.838781356811523
bench:  -4.814460754394531
Curry:  8.275822639465332
KD:  4.195781707763672
Lebron:  7.258570671081543
McCaw:  -1.4460735321044922

starters:  17.47191619873047
bench:  -4.729060173034668
Curry:  7.575850486755371
KD:  4.268922805786133
Lebron:  6.327521800994873
McCaw:  -1.0144696235656738

starters:  15.686284065246582
bench:  -5.6049323081970215
Curry:  7.663092613220215
KD:  5.007345676422119
Lebron:  5.950517177581787
McCaw:  -0.6758566498756409

starters:  12.619712829589844
bench:  -6.524082183837891
Curry:  7.511176586151123
KD:  3.91272234916687
Lebron:  5.500310897827148
McCaw:  -0.8624022006988525

starters:  10.66436958312


starters:  12.239136695861816
bench:  -6.923463821411133
Curry:  4.2301740646362305
KD:  5.1358962059021
Lebron:  4.363396167755127
McCaw:  -1.5377955436706543

starters:  11.124165534973145
bench:  -7.867138862609863
Curry:  5.202691078186035
KD:  5.251394748687744
Lebron:  3.9363648891448975
McCaw:  -1.031456470489502

starters:  16.877248764038086
bench:  -9.377962112426758
Curry:  5.293537139892578
KD:  6.0232930183410645
Lebron:  4.0631866455078125
McCaw:  0.044277578592300415

starters:  13.940811157226562
bench:  -9.674215316772461
Curry:  6.165121555328369
KD:  6.3590312004089355
Lebron:  3.6641592979431152
McCaw:  -1.6259479522705078

starters:  15.656835556030273
bench:  -5.981457710266113
Curry:  6.081237316131592
KD:  6.279087543487549
Lebron:  4.337670803070068
McCaw:  -1.6336251497268677

starters:  14.674813270568848
bench:  -6.591089725494385
Curry:  5.054089069366455
KD:  6.53096342086792
Lebron:  4.2853827476501465
McCaw:  -0.30954232811927795

starters:  14.94082260


starters:  17.556678771972656
bench:  -5.208771705627441
Curry:  6.873604774475098
KD:  5.532375335693359
Lebron:  6.637272834777832
McCaw:  -1.1522891521453857

starters:  20.209712982177734
bench:  -6.203851222991943
Curry:  6.941347599029541
KD:  5.124294281005859
Lebron:  5.857718467712402
McCaw:  -0.01776207983493805

starters:  20.725494384765625
bench:  -8.910253524780273
Curry:  6.441838264465332
KD:  5.360659599304199
Lebron:  5.158055305480957
McCaw:  -0.7751936316490173

starters:  19.628904342651367
bench:  -7.227339744567871
Curry:  5.374034881591797
KD:  4.5602030754089355
Lebron:  4.736279487609863
McCaw:  -0.7802519202232361

starters:  16.54185676574707
bench:  -10.544281959533691
Curry:  5.8555779457092285
KD:  4.575988292694092
Lebron:  4.73862886428833
McCaw:  -1.423751711845398

starters:  16.81944465637207
bench:  -13.662925720214844
Curry:  5.631836414337158
KD:  4.393291473388672
Lebron:  4.5272932052612305
McCaw:  -1.3620718717575073

starters:  16.49504089355


starters:  25.58156394958496
bench:  -5.936910629272461
Curry:  6.448827743530273
KD:  5.627287864685059
Lebron:  7.6999335289001465
McCaw:  0.3923535943031311

starters:  26.246652603149414
bench:  -8.006467819213867
Curry:  6.520275592803955
KD:  6.031720161437988
Lebron:  7.612049579620361
McCaw:  0.10329297184944153

starters:  24.725900650024414
bench:  -4.997739315032959
Curry:  7.386951923370361
KD:  6.1455512046813965
Lebron:  7.9418864250183105
McCaw:  0.42337071895599365

starters:  21.512964248657227
bench:  -9.631231307983398
Curry:  5.6023406982421875
KD:  6.096099376678467
Lebron:  7.0466694831848145
McCaw:  -0.90708327293396

starters:  25.576950073242188
bench:  -13.275742530822754
Curry:  6.221903324127197
KD:  6.4263176918029785
Lebron:  5.452756881713867
McCaw:  -0.8923680782318115

starters:  22.679317474365234
bench:  -13.368001937866211
Curry:  4.471867561340332
KD:  5.955393314361572
Lebron:  6.159568786621094
McCaw:  -2.0725584030151367

starters:  22.664171218


starters:  18.462995529174805
bench:  -4.063758850097656
Curry:  7.020558834075928
KD:  4.011606693267822
Lebron:  5.671751022338867
McCaw:  0.13723787665367126

starters:  19.204618453979492
bench:  -3.22619366645813
Curry:  6.255234718322754
KD:  4.193084239959717
Lebron:  5.415345668792725
McCaw:  -0.6508668065071106

starters:  20.64022445678711
bench:  -5.499360084533691
Curry:  7.353888034820557
KD:  2.8474245071411133
Lebron:  6.680479526519775
McCaw:  -1.0678631067276

starters:  16.68268585205078
bench:  -5.513077735900879
Curry:  5.860006809234619
KD:  3.5077285766601562
Lebron:  7.478066921234131
McCaw:  -0.6005550622940063

starters:  14.71921157836914
bench:  -3.385110378265381
Curry:  5.355088233947754
KD:  4.252514362335205
Lebron:  6.918990612030029
McCaw:  -1.941667079925537

starters:  15.519922256469727
bench:  -7.019452095031738
Curry:  5.08090353012085
KD:  5.330486297607422
Lebron:  5.753123760223389
McCaw:  -1.89206063747406

starters:  13.91532039642334
bench: 

In [ ]:
model.offensive_embeddings(torch.LongTensor([normal_name_to_idx['stephen curry']]).cuda())[0]

In [ ]:
print(losses)

In [ ]:
print([l/100000000 for l in losses])

In [ ]:
torch.FloatTensor(box)